##### データセットの場所やバッチサイズなどの定数値の設定

In [ ]:
import os
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'


# 使用するデバイス
# GPU を使用しない環境（CPU環境）で実行する場合は DEVICE = 'cpu' とする
DEVICE = 'cuda:0'

# 全ての訓練データを一回ずつ使用することを「1エポック」として，何エポック分学習するか
N_EPOCHS = 3

# 学習時のバッチサイズ
BATCH_SIZE = 100

# 訓練データセット（画像ファイルリスト）のファイル名
TRAIN_DATASET_CSV = './tinySTL10/train_list.csv'

# テストデータセット（画像ファイルリスト）のファイル名
TEST_DATASET_CSV = './tinySTL10/test_list.csv'

# 画像ファイルの先頭に付加する文字列（データセットが存在するディレクトリのパス）
DATA_DIR = './tinySTL10'

# 学習結果の保存先フォルダ
MODEL_DIR = './TL_models'

# 学習結果のニューラルネットワークの保存先
MODEL_FILE = os.path.join(MODEL_DIR, 'TL_object_recognizer_model.pth')

##### バックボーンモデルとして使用する ResNet18 のロード

In [ ]:
from torchvision import models
from torchvision.models import ResNet18_Weights

# ResNet18のモデルをロード
# 参考： https://pytorch.org/vision/stable/models.html
resnet = models.resnet18(weights=ResNet18_Weights.DEFAULT)

##### ResNet18のモデル構造の出力（転移学習に際し，どの層を出力を流用するかを決めるため，構造を確認する）

In [ ]:
print(resnet)

##### 流用する層の決定，および，その層の出力サイズの確認

In [ ]:
import torch
from torchvision.models.feature_extraction import create_feature_extractor


# 流用する層の名前（例えば 'layer4' を流用する場合）
layer_name = 'layer4'

# ResNet18の 'layer4' 層の出力を 'feature' という名前で取得できるようにし，
# それを backbone という変数に保存しておく
backbone = create_feature_extractor(resnet, {layer_name: 'feature'})

# 224x224 ピクセルのランダムなカラー画像を用意し，それを backbone に入力してみる．
# これにより，流用する層の出力サイズ（パーセプトロンがどのような並びになっているか）を調べる．
# [1, 512, 7, 7] のように出力されたら，サイズ 512x7x7 の特徴マップ（チャンネル数 512, 縦幅 7, 横幅 7 の特徴マップ）が出力される，ということ
x = torch.rand((1, 3, 224, 224), dtype=torch.float32)
h = backbone(x)['feature']
print(h.size())

##### ニューラルネットワークモデルの定義

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms


# 畳込み，バッチ正規化，ReLUをセットで行うクラス
class myConv2d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, padding):
        super(myConv2d, self).__init__()
        self.conv = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, stride=stride, padding=padding)
        self.bn = nn.BatchNorm2d(num_features=out_channels)
    def forward(self, x):
        return F.relu(self.bn(self.conv(x)))


# STL10物体画像認識AIを実現するニューラルネットワーク（転移学習バージョン）
class STL10Recognizer(nn.Module):

    # N: 認識対象となるクラスの数
    def __init__(self, N):
        super(STL10Recognizer, self).__init__()

        # 前処理
        self.preprocess = transforms.Compose([
            transforms.Resize(224, antialias=False), # 画像サイズを 224x224 ピクセルにリサイズする
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), # 取り出した部分の画素値を正規化する（ResNetと同じ条件にするため）
        ])

        # バックボーンモデルの登録
        self.backbone = backbone

        # バックボーンモデルのパラメータを固定（これをしない場合はファインチューニングになる）
        for param in self.backbone.parameters():
            param.requires_grad = False

        # 畳込み層
        # 例えば 'layer4' 層を流用する場合，その出力サイズが 512x7x7 なので，in_channels を 512 にする．
        # なお，その他の設定値が kernel_size=3, stride=1, padding=0 となっているので，この層を通した後のサイズは 64x5x5 になる
        self.conv = myConv2d(in_channels=512, out_channels=64, kernel_size=3, stride=1, padding=0)

        # 平坦化
        self.flat = nn.Flatten()

        # 全結合層
        self.fc1 = nn.Linear(in_features=64*5*5, out_features=256)
        self.fc2 = nn.Linear(in_features=256, out_features=N)

    def forward(self, x):
        h = self.preprocess(x)
        h = self.backbone(h)['feature']
        h = self.conv(h)
        h = self.flat(h)
        h = F.relu(self.fc1(h))
        y = self.fc2(h)
        return y

##### 訓練データセットの読み込み

In [ ]:
import pickle
from torch.utils.data import DataLoader, random_split
from mylib.data_io import CSVBasedDataset


# CSVファイルを読み込み, 訓練データセットを用意
dataset = CSVBasedDataset(
    filename = TRAIN_DATASET_CSV,
    items = [
        'File Path', # X
        'Class Name' # Y
    ],
    dtypes = [
        'image', # Xの型
        'label' # Yの型
    ],
    dirname = DATA_DIR,
    img_mode = 'color' # 強制的にカラー画像として読み込む
)
with open(os.path.join(MODEL_DIR, 'fdicts.pkl'), 'wb') as fdicts_file:
    pickle.dump(dataset.forward_dicts, fdicts_file)

# 認識対象のクラス数を取得
n_classes = len(dataset.forward_dicts[1])

# 訓練データセットを分割し，一方を検証用に回す
dataset_size = len(dataset)
valid_size = int(0.1 * dataset_size) # 全体の 10% を検証用に
train_size = dataset_size - valid_size # 残りの 90% を学習用に
train_dataset, valid_dataset = random_split(dataset, [train_size, valid_size])

# 訓練データおよび検証用データをミニバッチに分けて使用するための「データローダ」を用意
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, pin_memory=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False, pin_memory=True)

##### 学習処理の実行

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from mylib.visualizers import LossVisualizer


# ニューラルネットワークの作成
model = STL10Recognizer(N=n_classes).to(DEVICE)

# 最適化アルゴリズムの指定（ここでは SGD でなく Adam を使用）
optimizer = optim.Adam(model.parameters())

# 損失関数：クロスエントロピー損失を使用
loss_func =  nn.CrossEntropyLoss()

# 損失関数値を記録する準備
loss_viz = LossVisualizer(['train loss', 'valid loss'])

# 勾配降下法による繰り返し学習
for epoch in range(N_EPOCHS):

    print('Epoch {0}:'.format(epoch + 1))

    # 学習
    model.train()
    model.backbone.eval() # ファインチューニングの場合は不要
    sum_loss = 0
    for X, Y in tqdm(train_dataloader): # X, Y は CSVBasedDataset クラスの __getitem__ 関数の戻り値に対応
        for param in model.parameters():
            param.grad = None
        X = X.to(DEVICE)
        Y = Y.to(DEVICE)
        Y_pred = model(X) # 入力画像 X を現在のニューラルネットワークに入力し，出力の推定値を得る
        loss = loss_func(Y_pred, Y) # 損失関数の現在値を計算
        loss.backward() # 誤差逆伝播法により，個々のパラメータに関する損失関数の勾配（偏微分）を計算
        optimizer.step() # 勾配に沿ってパラメータの値を更新
        sum_loss += float(loss) * len(X)
    avg_loss = sum_loss / train_size
    loss_viz.add_value('train loss', avg_loss) # 訓練データに対する損失関数の値を記録
    print('train loss = {0:.6f}'.format(avg_loss))

    # 検証
    model.eval()
    sum_loss = 0
    n_failed = 0
    with torch.inference_mode():
        for X, Y in tqdm(valid_dataloader):
            X = X.to(DEVICE)
            Y = Y.to(DEVICE)
            Y_pred = model(X)
            loss = loss_func(Y_pred, Y)
            sum_loss += float(loss) * len(X)
            n_failed += torch.count_nonzero(torch.argmax(Y_pred, dim=1) - Y) # 推定値と正解値が一致していないデータの個数を数える
    avg_loss = sum_loss / valid_size
    loss_viz.add_value('valid loss', avg_loss) # 検証用データに対する損失関数の値を記録
    accuracy = (valid_size - n_failed) / valid_size
    print('valid loss = {0:.6f}'.format(avg_loss))
    print('accuracy = {0:.2f}%'.format(100 * accuracy))
    print('')

# 学習結果のニューラルネットワークモデルをファイルに保存
model = model.to('cpu')
torch.save(model.state_dict(), MODEL_FILE)

# 損失関数の記録をファイルに保存
loss_viz.save(v_file=os.path.join(MODEL_DIR, 'loss_graph.png'), h_file=os.path.join(MODEL_DIR, 'loss_history.csv'))

##### テストデータセットの読み込み

In [ ]:
import pickle
from torch.utils.data import DataLoader
from mylib.data_io import CSVBasedDataset


# CSVファイルを読み込み, テストデータセットを用意
with open(os.path.join(MODEL_DIR, 'fdicts.pkl'), 'rb') as fdicts_file:
    fdicts = pickle.load(fdicts_file)
test_dataset = CSVBasedDataset(
    filename = TEST_DATASET_CSV,
    items = [
        'File Path', # X
        'Class Name' # Y
    ],
    dtypes = [
        'image', # Xの型
        'label' # Yの型
    ],
    dirname = DATA_DIR,
    img_mode = 'color', # 強制的にカラー画像として読み込む
    fdicts = fdicts
)
test_size = len(test_dataset)
rdict = test_dataset.reverse_dicts[1]

# 認識対象のクラス数を取得
n_classes = len(test_dataset.forward_dicts[1])

# テストデータをミニバッチに分けて使用するための「データローダ」を用意
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, pin_memory=True)

##### 学習済みニューラルネットワークモデルのロード

In [ ]:
import torch


# ニューラルネットワークモデルとその学習済みパラメータをファイルからロード
model = STL10Recognizer(N=n_classes)
model.load_state_dict(torch.load(MODEL_FILE))

##### 単一画像に対するテスト処理の実行

In [ ]:
import torch
from mylib.data_io import show_single_image


model = model.to(DEVICE)
model.eval()

# index 番目のテストデータをニューラルネットワークに入力してみる
while True:
    print('index?: ', end='')
    val = input()
    if val == 'exit': # 'exit' とタイプされたら終了
        break
    index = int(val)
    x, y = test_dataset[index]
    x = x.reshape(1, *x.size()).to(DEVICE)
    with torch.inference_mode():
        y_pred = model(x)
    y_pred = torch.argmax(y_pred, dim=1)
    print('')
    print('estimated:', rdict[int(y_pred)])
    print('ground truth:', rdict[int(y)])
    print('')
    show_single_image(x.to('cpu'), title='input image', sec=1)

##### 全ての画像に対するテスト処理の実行

In [ ]:
import torch
from tqdm import tqdm


model = model.to(DEVICE)
model.eval()

# テストデータセットを用いて認識精度を評価
n_failed = 0
with torch.inference_mode():
    for X, Y in tqdm(test_dataloader):
        X = X.to(DEVICE)
        Y = Y.to(DEVICE)
        Y_pred = model(X)
        n_failed += torch.count_nonzero(torch.argmax(Y_pred, dim=1) - Y) # 推定値と正解値が一致していないデータの個数を数える
    accuracy = (test_size - n_failed) / test_size
    print('accuracy = {0:.2f}%'.format(100 * accuracy))
    print('')